In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Datasets
dataset_root = './data'
trainset = datasets.CIFAR100(dataset_root, train=True, transform=data_transforms, download=True)
testset = datasets.CIFAR100(dataset_root, train=False, transform=data_transforms, download=True)

# DataLoaders
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

# Define custom CNN
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomCNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop with time and progress tracking
num_epochs = 30
best_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    start_time = time.time()
    running_loss = 0.0
    model.train()

    total_batches = len(trainloader)
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate and print progress percentage
        progress = (i + 1) / total_batches * 100
        print(f'\rProgress: {progress:.2f}% [{i+1}/{total_batches}]', end='')

    elapsed_time = time.time() - start_time
    print(f'\nEpoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(trainloader)}, Time: {elapsed_time:.2f}s')

    if running_loss < best_loss:
        best_loss = running_loss
        torch.save(model.state_dict(), 'custom_best_model.pth')

# Load the best model for evaluation
model.load_state_dict(torch.load('custom_best_model.pth'))
model.eval()

# Testing loop with time and progress tracking
correct = 0
total = 0
start_time = time.time()

total_batches = len(testloader)
with torch.no_grad():
    for i, (inputs, labels) in enumerate(testloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # Calculate and print progress percentage
        progress = (i + 1) / total_batches * 100
        print(f'\rTesting Progress: {progress:.2f}% [{i+1}/{total_batches}]', end='')

elapsed_time = time.time() - start_time
accuracy = correct / total * 100
print(f'\nAccuracy on the test set: {accuracy:.2f}%, Time: {elapsed_time:.2f}s')

Using device: cuda:0


100%|██████████| 169M/169M [00:05<00:00, 31.0MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1/30
Progress: 100.00% [1563/1563]
Epoch [1/30], Loss: 4.261129578183419, Time: 25.27s
Epoch 2/30
Progress: 100.00% [1563/1563]
Epoch [2/30], Loss: 3.622346077221598, Time: 24.11s
Epoch 3/30
Progress: 100.00% [1563/1563]
Epoch [3/30], Loss: 3.2488017314073754, Time: 23.96s
Epoch 4/30
Progress: 100.00% [1563/1563]
Epoch [4/30], Loss: 2.9731467734219095, Time: 24.54s
Epoch 5/30
Progress: 100.00% [1563/1563]
Epoch [5/30], Loss: 2.747159311425129, Time: 24.09s
Epoch 6/30
Progress: 100.00% [1563/1563]
Epoch [6/30], Loss: 2.54673184443954, Time: 24.22s
Epoch 7/30
Progress: 100.00% [1563/1563]
Epoch [7/30], Loss: 2.38362443782699, Time: 24.69s
Epoch 8/30
Progress: 100.00% [1563/1563]
Epoch [8/30], Loss: 2.231063853146095, Time: 23.74s
Epoch 9/30
Progress: 100.00% [1563/1563]
Epoch [9/30], Loss: 2.0893292725276886, Time: 23.76s
Epoch 10/30
Progress: 100.00% [1563/1563]
Epoch [10/30], Loss: 1.9546411

<ipython-input-1-288aa3c0a147>:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('custom_best_model.pth'))


Testing Progress: 100.00% [313/313]
Accuracy on the test set: 47.21%, Time: 4.06s
